In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie

  # CairoMakie's display() in PDF format opens an interactive window
  # instead of saving to the ipynb file, so we don't do that.
  # https://github.com/quarto-dev/quarto-cli/issues/7548
  if fig_format == :pdf
    CairoMakie.activate!(type = "png")
  else
    CairoMakie.activate!(type = string(fig_format))
  end
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/Users/hirofumi48/162348.github.io/posts/2025/Comp"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| output: false
n, p, pₑ = 200, 50, 10

using Random, StatsFuns, Distributions
β_true = vcat(randn(pₑ), zeros(p - pₑ))
X = randn(n, p)

η_true = X * β_true
π_true = logistic.(η_true)

y = rand.(Bernoulli.(π_true))
y = collect(Float64, y)

200-element Vector{Float64}:
 1.0
 1.0
 0.0
 1.0
 0.0
 1.0
 0.0
 1.0
 1.0
 1.0
 1.0
 0.0
 1.0
 ⋮
 1.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 1.0
 0.0

In [3]:
#| output: false
using PolyaGammaHybridSamplers, LinearAlgebra, MCMCChains, Dates, MCMCDiagnosticTools

function pg_logistic_gibbs(
  X::Matrix{Float64},
  y::Vector{Float64};
  n_iter::Int = 5000,
  burnin::Int = 1000,
  σ_prior::Float64 = 10.0,
)
  n, p = size(X)

  # 事前: β ~ N(0, σ_prior^2 I)
  V0_inv = (1 / σ_prior^2) * LinearAlgebra.I  # precision of prior

  # 初期値
  β = zeros(p)
  κ = y .- 0.5  # κ_i = y_i - 1/2

  # サンプル保存用
  n_save = n_iter - burnin
  β_samples = Matrix{Float64}(undef, n_save, p)

  t_start = time()
  for it in 1:n_iter
    # 1. PG 補助変数 ω_i | β のサンプル
    η = X * β
    ω = similar(η)
    for i in 1:n
      pg = PolyaGammaHybridSampler(1.0, η[i])
      ω[i] = rand(pg)
    end

    # 2. β | ω, y のサンプル (多変量ガウス)
    Ω = Diagonal(ω)
    precision = X' * Ω * X + V0_inv          # posterior precision
    cov = inv(Matrix(precision))             # posterior covariance
    m = cov * (X' * κ)                       # posterior mean (μ0=0 のため)

    # β ~ N(m, cov)
    β = rand(MvNormal(m, Symmetric(cov)))

    # burn-in 後に保存
    if it > burnin
      β_samples[it - burnin, :] .= β
    end
  end
  t_stop = time()
  runtime_sec = t_stop - t_start

  names = Symbol.("β[$i]" for i in 1:p)
  values = reshape(β_samples, n_save, p, 1)
  chain = Chains(values, names)
  chain = setinfo(chain, (
    start_time = [t_start],  # 1本チェインなら長さ1のベクトルでOK
    stop_time  = [t_stop],
  ))

  return chain, runtime_sec
end

pg_logistic_gibbs (generic function with 1 method)

In [4]:
σ_prior = 10.0
chain_pg, t_pg = pg_logistic_gibbs(X, y;
    n_iter = 3000,
    burnin = 3000,
    σ_prior = σ_prior,
)
summarize(chain_pg)

┌ Warning: number of draws after splitting must be >4 but is 0. ESS cannot be computed.
└ @ MCMCDiagnosticTools ~/.julia/packages/MCMCDiagnosticTools/x6KGX/src/ess_rhat.jl:472
┌ Warning: number of draws after splitting must be >4 but is 0. ESS cannot be computed.
└ @ MCMCDiagnosticTools ~/.julia/packages/MCMCDiagnosticTools/x6KGX/src/ess_rhat.jl:472


┌ Warning: number of draws after splitting must be >4 but is 0. ESS cannot be computed.
└ @ MCMCDiagnosticTools ~/.julia/packages/MCMCDiagnosticTools/x6KGX/src/ess_rhat.jl:472
┌ Warning: number of draws after splitting must be >4 but is 0. ESS cannot be computed.
└ @ MCMCDiagnosticTools ~/.julia/packages/MCMCDiagnosticTools/x6KGX/src/ess_rhat.jl:472




  parameters      mean       std      mcse   ess_bulk   ess_tail      rhat   e ⋯
      Symbol   Float64   Float64   Float64    Float64    Float64   Float64     ⋯

        β[1]       NaN       NaN       NaN        NaN        NaN    0.0000     ⋯
        β[2]       NaN       NaN       NaN        NaN        NaN    0.0000     ⋯
        β[3]       NaN       NaN       NaN        NaN        NaN    0.0000     ⋯
        β[4]       NaN       NaN       NaN        NaN        NaN    0.0000     ⋯
        β[5]       NaN       NaN       NaN        NaN        NaN    0.0000     ⋯
        β[6]       NaN       NaN       NaN        NaN        NaN    0.0000     ⋯
        β[7]       NaN       NaN       NaN        NaN        NaN    0.0000     ⋯
        β[8]       NaN       NaN       NaN        NaN        NaN    0.0000     ⋯
        β[9]       NaN       NaN       NaN        NaN        NaN    0.0000     ⋯
       β[10]       NaN       NaN       NaN        NaN        NaN    0.0000     ⋯
       β[11]       NaN   

In [5]:
#| output: false
using Turing, LinearAlgebra

@model function logreg_turing(x, y, σ_prior)
    n, p = size(x)
    
    # 事前分布
    β ~ MvNormal(zeros(p), (σ_prior^2) * I)
    
    # ベクトル化した尤度（高速化）
    η = x * β
    y ~ arraydist(Bernoulli.(logistic.(η)))
end

model = logreg_turing(X, y, σ_prior)

DynamicPPL.Model{typeof(logreg_turing), (:x, :y, :σ_prior), (), (), Tuple{Matrix{Float64}, Vector{Float64}, Float64}, Tuple{}, DynamicPPL.DefaultContext}(Main.logreg_turing, (x = [-0.33630482063128264 0.49085612781842686 … 0.5007744191204583 -1.134508218851494; -2.2370926068010375 -1.8840864082071576 … 0.6869180147505104 -1.6923355054471574; … ; -0.3228216908663224 0.10719559504403786 … 0.9198383954789953 0.43246054034653303; 0.37190409068235114 1.5603736650414053 … 0.6759677053413145 0.11486773803956256], y = [1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0  …  0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0], σ_prior = 10.0), NamedTuple(), DynamicPPL.DefaultContext())

In [6]:
#| output: false
n_samples = 3000
n_adapt   = 3000

chain_hmc = sample(
    model,
    NUTS(n_adapt, 0.6),
    n_samples,
)

┌ Info: Found initial step size
└   ϵ = 0.8


Sampling:   2%|▋                                        |  ETA: 0:01:04

Sampling:   2%|▉                                        |  ETA: 0:01:11

Sampling:   2%|█                                        |  ETA: 0:01:14

Sampling:   3%|█▎                                       |  ETA: 0:01:16

...

Sampling:  98%|████████████████████████████████████████▍|  ETA: 0:00:01

Sampling:  99%|████████████████████████████████████████▋|  ETA: 0:00:01

Sampling:  99%|████████████████████████████████████████▊|  ETA: 0:00:00

Sampling: 100%|█████████████████████████████████████████| Time: 0:01:03

Chains MCMC chain (3000×64×1 Array{Float64, 3}):

Iterations        = 3001:1:6000
Number of chains  = 1
Samples per chain = 3000
Wall duration     = 64.56 seconds
Compute duration  = 64.56 seconds
parameters        = β[1], β[2], β[3], β[4], β[5], β[6], β[7], β[8], β[9], β[10], β[11], β[12], β[13], β[14], β[15], β[16], β[17], β[18], β[19], β[20], β[21], β[22], β[23], β[24], β[25], β[26], β[27], β[28], β[29], β[30], β[31], β[32], β[33], β[34], β[35], β[36], β[37], β[38], β[39], β[40], β[41], β[42], β[43], β[44], β[45], β[46], β[47], β[48], β[49], β[50]
internals         = n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size, lp, logprior, loglikelihood

Use `describe(chains)` for summary statistics and quantiles.


In [7]:
summarize(chain_hmc)



  parameters       mean       std      mcse    ess_bulk    ess_tail      rhat  ⋯
      Symbol    Float64   Float64   Float64     Float64     Float64   Float64  ⋯

        β[1]   -21.2574    2.9705    0.0935   1011.2394   1351.3094    1.0001  ⋯
        β[2]    -2.8056    1.7852    0.0437   1668.3259   1487.6221    1.0000  ⋯
        β[3]     9.1332    2.3617    0.0613   1489.4921   1859.1893    1.0009  ⋯
        β[4]    -2.3807    2.2998    0.0569   1635.9912   1707.0213    0.9997  ⋯
        β[5]    26.1547    3.4396    0.1026   1137.2876   1498.4444    1.0007  ⋯
        β[6]   -26.7011    3.3202    0.1027   1040.8542   1589.5197    1.0008  ⋯
        β[7]   -10.9273    2.4249    0.0708   1182.1433   1602.7193    1.0015  ⋯
        β[8]   -10.2274    2.3938    0.0598   1616.5992   2125.6429    1.0005  ⋯
        β[9]    38.2108    4.5591    0.1375   1109.9808   1651.6727    1.0010  ⋯
       β[10]   -12.6827    2.6123    0.0735   1264.5977   1747.1909    1.0002  ⋯
       β[11]     5.8228  

In [8]:
using Statistics

# 真の β との誤差
mean_hmc = vec(mean(Array(chain_hmc), dims=1))  # ここは実際のパラメータ名に合わせて調整
mean_pg = vec(mean(Array(chain_pg), dims=1))

println("‖β̂_HMC - β_true‖₂ = ", norm(mean_hmc .- β_true))
println("‖β̂_PG  - β_true‖₂ = ", norm(mean_pg  .- β_true))

# ランタイムや ESS の比較も：
ess_hmc = ess_rhat(chain_hmc)
ess_pg  = ess_rhat(chain_pg)
println("ESS/s (HMC) = ", mean(ess_hmc[:,:ess_per_sec]))
println("ESS/s (PG) = ", mean(ess_pg[:,:ess_per_sec]))

‖β̂_HMC - β_true‖₂ = 62.896105198368296
‖β̂_PG  - β_true‖₂ = NaN
ESS/s (HMC) = 23.573264077259456

┌ Warning: number of draws after splitting must be >4 but is 0. ESS cannot be computed.
└ @ MCMCDiagnosticTools ~/.julia/packages/MCMCDiagnosticTools/x6KGX/src/ess_rhat.jl:472



ESS/s (PG) = NaN
